# Udacity Data Wrangling Project (SQL)
### Justin "Roy" Garrard | 07/19/17

#### Introduction

This project is intended to demonstrate an understanding of data cleaning processes. A subset of OpenStreetMap data will be cleaned, converted into a .csv file, and imported into a SQL database.

#### Overview

1. Audit the Data
2. Converting the Data
3. Exploring the Data

#### About the Data

* Contains the Idaho cities Boise, Meridian, Nampa, Eagle, Kuna, and Caldwell. 
* A rectangular section bounded by the coordinates (43.4596,-116.7517,43.7552,-115.9765).
* Roughly 68 MB in size.
* Has 455 unique users.
* Has 281,196 nodes and 33,401 ways.
* Contains 165 schools.
* Contains 20 cafes.
* Contains 13 Subway sandwhich shops.
* Has undergone 314,597 modifications since its beginning in 2006.

In [1]:
# Imported Libraries
import xml.etree.cElementTree as ET   # XML Processing
from collections import defaultdict   # Hashmap w/ Default Value
import pprint                         # Improved print 
import re                             # Regular Expressions
import csv                            # CSV Handler
import codecs                         # File Opener
import sqlite3                        # SQL Interface

In [2]:
# Useful Regular Expressions
letter = re.compile(r'[a-z]', re.IGNORECASE)
string = re.compile(r'[^0-9\.\-]')
apo = re.compile(r"'")
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
problemchars_x = re.compile(r'[=\+/&<>;\'"\?%#$@\,\.\t\r\n]') # Excludes spaces
double_space = re.compile(r'  ')
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
street_dir_re = re.compile(r'\b[a-z]\b', re.IGNORECASE)

In [3]:
# Load in Data
osm_file = open('idaho_sw.xml', "r")

### Audit the Data (XML)

**Street Data**

In [4]:
# audit_streets Street Names  
'''
Determines if a provided node contains street name data.

elem: An ET node.
returns: Boolean.
'''
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

'''
Scans the OpenStreetMap file for all recorded street suffixes.

street_list: A list of street names.
returns: A defaultdict containing all street suffixes as keys
and instances of streets using those suffixes as values.
'''
def audit_streets(street_list):
    street_types = defaultdict(set)
    for item in street_list:
        m = street_type_re.search(item)
        if m:
            street_type = m.group()
            street_types[street_type].add(item)
    return street_types

# Run through the OSM data to collect data
street_list = []
for event, elem in ET.iterparse(osm_file, events=("start",)):
    if elem.tag == "node" or elem.tag == "way":
        for tag in elem.iter("tag"):
            if is_street_name(tag):
                street_list.append(tag.attrib['v'])
                
# audit_streets the OSM data
street_types = audit_streets(street_list)
pprint.pprint(street_types)

defaultdict(<class 'set'>,
            {'320': {'W. Main Street, STE 320'},
             'A': {'West Fairview Avenue #A'},
             'Ave': {'E Maine Ave', 'W Fairview Ave', 'W Taylor Ave'},
             'Ave.': {'West Fairview Ave.'},
             'Avenue': {'1021 Broadway Avenue',
                        'Arlington Avenue',
                        'Broadway Avenue',
                        'Creekside Avenue',
                        'East Boise Avenue',
                        'East Broadway Avenue',
                        'East Hawaii Avenue',
                        'East Roosevelt Avenue',
                        'Fairview Avenue',
                        'Florida Avenue',
                        'Georgia Avenue',
                        'Hawkins Avenue',
                        'Iowa Avenue',
                        'Lake Lowell Avenue',
                        'Law Avenue',
                        'Lincoln Avenue',
                        'Manitou Avenue',
                  

In [5]:
# Number of streets
# (change previous cell from set to list for unique/non-unique)
counter = 0
for street_list in street_types.values():
    for street in street_list:
        counter += 1

print(counter)

362


**Author Note**

* The street data is surpisingly clean for containing six (albeit small) cities. Either the city planners know their stuff, or someone's already been over this data.

* There are 3307 street instances in this data set, but only 362 unique street names. 

* Given the size of the data, I suspect that only main roads were included. Many neighborhood streets and similarly minor paths do not appear to be in the data set.

In [6]:
# Clean Street Names
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", 
            "Place", "Square", "Lane", "Road", "Trail", 
            "Parkway", "Commons"]

suffix_mapping = { "St": "Street",
            "St.": "Street",
            "Rd": "Road",
            "Rd.": "Road",
            "RD": "Road",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Ln": "Lane",
            "Ln.": "Lane",
            "Dr": "Drive",
            }

prefix_mapping = { "N.": "North",
            "N": "North",
            "S.": "South",
            "S": "South",
            "W.": "West",
            "W": "West",
            "E.": "East",
            "E": "East",
}

'''
Returns a cleaned version of the input street name.

name: A string representing a street name.
returns: String.
'''
def update_street_name(name):
    fix = name.split()
    first = True
    for word in fix:
        if first and 'Mc' not in word and 'ID' not in word:
            name = name.replace(word, word.capitalize())
            word = word.capitalize()
            first = False
        if word in suffix_mapping.keys():
            name = name.replace(word, suffix_mapping[word])
        elif word in prefix_mapping.keys() and len(word) <= 2:
            name = name.replace(word, prefix_mapping[word])
    name = name.replace('.', '')
    return name

'''
Tests the effectiveness of update_street_name by printing changes.

street_types: A dictionary of street names by prefix.
'''
def test_clean(street_types):
    clean = defaultdict(set)
    for st_type, ways in street_types.items():
        for name in ways:
            better_name = update_street_name(name)
            if better_name != name:
                print(name + ' -> ' + better_name)


In [7]:
test_clean(street_types)

W. Grove Street -> West Grove Street
106 S. 11th Street -> 106 South 11th Street
N 4th Street -> North 4th Street
E. State Street -> East State Street
W. Main Street -> West Main Street
W. Overland Road -> West Overland Road
#110West Overland Road -> #110west Overland Road
S. Five Mile Road -> South Five Mile Road
W. Chinden Boulevard -> West Chinden Boulevard
N. Linder Rd. -> North Linder Road
north Kings Rd. -> North Kings Road
E Maine Ave -> East Maine Avenue
W Fairview Ave -> West Fairview Avenue
W Taylor Ave -> West Taylor Avenue
Emerald St -> Emerald Street
W Main St -> West Main Street
W Washington St -> West Westashington Street
888 W Fort St -> 888 West Fort Street
B Ellen St -> B Ellen Street
W Bethel Court -> West Bethel Court
S. Eckert -> South Eckert
N Lakeharbor Ln -> North Lakeharbor Lane
12th Ave Rd #B -> 12th Avenue Road #B
Caldwell Blvd -> Caldwell Boulevard
E Caldwell Blvd -> East Caldwell Boulevard
W. Main Street, STE 320 -> West Main Street, STE 320
Tango Creek Dr 

**Postal Codes**

In [8]:
# Audit postal codes
'''
Determines if a provided node contains postal code data.

elem: An ET node.
returns: Boolean.
'''
def is_post_code(elem):
    return ("post" in elem.attrib['k'])

# Refresh file, because XML parse is weird
osm_file.close()
osm_file = open('idaho_sw.xml', "r")

# Run through the OSM data to collect data
post_list = defaultdict(int)
for event, elem in ET.iterparse(osm_file, events=("start",)):
    if elem.tag == "node" or elem.tag == "way":
        for tag in elem.iter("tag"):
            if is_post_code(tag):
                post_list[tag.attrib['v']] += 1

pprint.pprint(post_list)

defaultdict(<class 'int'>,
            {'83605': 7,
             '83605-5407': 2,
             '83607': 2,
             '83616': 6,
             '83634': 6,
             '83642': 56,
             '83644': 3,
             '83646': 18,
             '83651': 559,
             '83651-8149': 28,
             '83669': 6,
             '83678': 1,
             '83686': 2223,
             '83687': 172,
             '83701': 1,
             '83702': 26,
             '83702-5625': 1,
             '83703': 7,
             '83704': 39,
             '83705': 30,
             '83706': 30,
             '83709': 19,
             '83712': 8,
             '83713': 5,
             '83714': 41,
             '83716': 4,
             '83735': 1,
             'ID 83702': 1,
             'Interstate 184  Boise, ID 83702': 1})


**Author's Note**

Not bad, just a few oddities that shouldn't be much trouble to fix.

In [9]:
# Clean postal codes
'''
Returns a cleaned version of the input postal code.

name: A string representing a street name.
returns: String.
'''
def update_post_code(code):
    code = re.sub(letter, '', code)
    code = re.sub(problemchars, ' ', code)
    code = re.sub(double_space, '', code)
    code = code.strip()
    fix = code.split('-')
    return fix[0]

'''
Tests the effectiveness of update_post_name by printing changes.

post_codes: A dictionary of postal codes.
'''
def test_post_clean(post_codes):
    clean = defaultdict(set)
    for code, count in post_codes.items():
        better_code = update_post_code(code)
        if better_code != code:
            print(code + ' -> ' + better_code)

In [10]:
test_post_clean(post_list)

Interstate 184  Boise, ID 83702 -> 184 83702
ID 83702 -> 83702
83651-8149 -> 83651
83702-5625 -> 83702
83605-5407 -> 83605


**Author's Note**

Not a complete success, but pretty close. It would be a little ridiculous to build a cleaner that can handle the single 'Interstate 184' entry. Better to just remove it by hand.

### Converting the Data (XML -> CSV)

In [11]:
# Set path names, regex, and fields
OSM_PATH = "idaho_sw.xml"
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

In [12]:
# Functions for the conversion (modified from the Udacity exercise)
'''
Clean and shape node or way XML element to Python dict

element: XML element.
node_attr: Schema of fields for node element.
way_attr_fields: Schema of fields for way element.
problem_chars: A regex for finding problem characters.
returns: A dictionary with formatted data relevant to the element type.
'''
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=problemchars):

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        for field in node_attr_fields:
            node_attribs[field] = element.attrib[field] 
        
        node_id = node_attribs['id']
        tag_iterator = element.iter("tag")
        tags = process_tags(tag_iterator, node_id)        
        return {'node': node_attribs, 'node_tags': tags}
        
    elif element.tag == 'way':
        for field in way_attr_fields:
            way_attribs[field] = element.attrib[field] 
        
        way_id = way_attribs['id']
        position = 0
        for node_element in element.iter("nd"):
            node_dict = {}
            node_dict['id'] = way_id
            node_dict['node_id'] = node_element.attrib['ref']
            node_dict['position'] = position
            position += 1
            way_nodes.append(node_dict)
        
        tag_iterator = element.iter("tag")
        tags = process_tags(tag_iterator, way_id)        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
'''
Yield element if it is the right type of tag.

osm_file: The filename for the XML data.
tags: List of tags.
'''
def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

'''
Process tag elements, cleaning and organizing them into a dict.

tag_iterator: An iterator for the tag elements.
default_tag_type: The default value for tags without a declared type.
returns: A dictionary with tag attributes.
'''
def process_tags(tag_iterator, node_id, default_tag_type='regular'):
    tags = []
    for tag_element in tag_iterator:
            if not re.match(problemchars, tag_element.attrib['k']):
                tag_dict = {}
                tag_dict['id'] = node_id
                if re.match(lower_colon, tag_element.attrib['k']):
                    index = tag_element.attrib['k'].find(':')
                    tag_dict['key'] = tag_element.attrib['k'][index+1:]
                    tag_dict['type'] = tag_element.attrib['k'][:index]
                    if is_street_name(tag_element):
                        tag_dict['value'] = update_street_name(tag_element.attrib['v'])
                    elif is_post_code(tag_element):
                        tag_dict['value'] = update_post_code(tag_element.attrib['v'])
                    else:
                        tag_dict['value'] = tag_element.attrib['v']
                else:
                    tag_dict['key'] = tag_element.attrib['k']
                    tag_dict['type'] = default_tag_type
                    tag_dict['value'] = tag_element.attrib['v']
                tags.append(tag_dict)
    return tags

# ================================================== #
#               Main Function                        #
# ================================================== #
'''
Iteratively process each XML element and write to csv(s).

file_in: The input filename.
'''
def process_map(file_in):
    with codecs.open(NODES_PATH, 'w') as nodes_file, \
        codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
        codecs.open(WAYS_PATH, 'w') as ways_file, \
        codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
        codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = csv.DictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = csv.DictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = csv.DictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = csv.DictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = csv.DictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])



In [13]:
# Run the cleaning/conversion
process_map(OSM_PATH)

### Storing the Data (SQL)

In [14]:
# Establish database connection
con = sqlite3.connect("osm_idaho.db")
cur = con.cursor()

In [15]:
# SQL Table Creation Commands
nodes_cmd = """
CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);
"""

nodes_tags_cmd = """
CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id)
);
"""

ways_cmd = """
CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT
);
"""

ways_tags_cmd = """
CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id)
);
"""

ways_nodes_cmd = """
CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id)
);
"""

In [16]:
# Drop any pre-existing tables
cur.execute("DROP TABLE IF EXISTS Nodes")
cur.execute("DROP TABLE IF EXISTS Nodes_Tags")
cur.execute("DROP TABLE IF EXISTS Ways")
cur.execute("DROP TABLE IF EXISTS Ways_Tags")
cur.execute("DROP TABLE IF EXISTS Ways_Nodes")

# Add fresh tables
cur.execute(nodes_cmd)
cur.execute(nodes_tags_cmd)
cur.execute(ways_cmd)
cur.execute(ways_tags_cmd)
cur.execute(ways_nodes_cmd)

In [17]:
# Fill tables with data
OSM_PATH = "idaho_sw.xml"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

'''
Fills the sql tables with data. Performs minor cleaning.

source: A csv reader object that outputs rows of data.
table_name: The corresponding table name, String.
'''
def fill_table(source, table_name):
    first = True
    col_id = ''
    err = []
    for row in source:
        if first:
            # Record header
            first = False
            col_id = '(' + ','.join(row) + ')'
            print(col_id)
        else:
            # Handle text elements
            for i in range(0,len(row)):
                row[i] = re.sub(problemchars_x, '', row[i])
                row[i] = re.sub(double_space, ' ', row[i])
                if re.search(string, row[i]):
                    row[i] = "'" + row[i] + "'"
            # Insert element into table
            cmd = "INSERT INTO " + table_name + col_id + " VALUES(" + ','.join(row) + ");"
            try:
                cur.execute(cmd)
            except:
                err.append(cmd)

# Open the csv's and start porting over the data
with open(NODES_PATH, 'r') as nodes_file, \
    open(NODE_TAGS_PATH, 'r') as node_tags_file, \
    open(WAYS_PATH, 'r') as ways_file, \
    open(WAY_TAGS_PATH, 'r') as way_tags_file, \
    open(WAY_NODES_PATH, 'r') as way_nodes_file:
        
        node_f = csv.reader(nodes_file)
        node_tags_f = csv.reader(node_tags_file)
        ways_f = csv.reader(ways_file)
        way_tag_f = csv.reader(way_tags_file)
        way_nodes_f = csv.reader(way_nodes_file)
        
        fill_table(node_f, "Nodes")
        fill_table(node_tags_f, "Nodes_Tags")
        fill_table(ways_f, "Ways")
        fill_table(way_tag_f, "Ways_Tags")
        fill_table(way_nodes_f, "Ways_Nodes") 
        

(id,lat,lon,user,uid,version,changeset,timestamp)
(id,key,value,type)
(id,user,uid,version,changeset,timestamp)
(id,key,value,type)
(id,node_id,position)


### Exploring the Data

In [18]:
# Determine number of nodes and ways
cur.execute("SELECT * FROM Nodes")
x = cur.fetchall()
cur.execute("SELECT * FROM Ways")
y = cur.fetchall()
print("Number of Nodes: " + str(len(x)))
print("Number of Ways: " + str(len(y)))

Number of Nodes: 281196
Number of Ways: 33401


In [19]:
# Determine number of unique user ID's
cur.execute("SELECT DISTINCT uid FROM Nodes UNION \
SELECT DISTINCT uid FROM Ways")
x = cur.fetchall()
print("Number of Unique User ID's: " + str(len(x)))

Number of Unique User ID's: 455


In [20]:
# Determine number of various places
amenities = defaultdict(int)
cur.execute("SELECT value,COUNT(value) FROM Nodes_Tags WHERE key IS 'amenity' \
GROUP BY value ORDER BY COUNT(value) DESC")
x = cur.fetchall()

pprint.pprint(x)

[('restaurant', 166),
 ('school', 165),
 ('place_of_worship', 93),
 ('fast_food', 82),
 ('fuel', 62),
 ('bank', 57),
 ('parking', 25),
 ('cafe', 20),
 ('bar', 13),
 ('pharmacy', 13),
 ('car_wash', 11),
 ('cinema', 11),
 ('doctors', 11),
 ('grave_yard', 10),
 ('hospital', 9),
 ('toilets', 9),
 ('bicycle_repair_station', 8),
 ('fire_station', 8),
 ('library', 8),
 ('post_office', 8),
 ('dentist', 7),
 ('drinking_water', 7),
 ('veterinary', 7),
 ('bench', 5),
 ('pub', 5),
 ('theatre', 5),
 ('parking_entrance', 4),
 ('public_building', 4),
 ('fountain', 3),
 ('shelter', 3),
 ('social_facility', 3),
 ('atm', 2),
 ('car_rental', 2),
 ('charging_station', 2),
 ('pool', 2),
 ('recycling', 2),
 ('townhall', 2),
 ('animal_shelter', 1),
 ('arts_centre', 1),
 ('bbq', 1),
 ('boat_rental', 1),
 ('nightclub', 1),
 ('police', 1),
 ('post_box', 1),
 ('prison', 1),
 ('swimming_pool', 1),
 ('telephone', 1)]


**Author's Note**

Speaking from personal experience, this data is far too sparse to represent Southwest Idaho. Some values seem reasonable (schools, libraries), some are questionable (bars, resturants), and some are obviously wrong (fuel, doctors, dentists).

In [21]:
# Investigate resturaunts
food_ids= []
food_places = defaultdict(int)
cur.execute("SELECT DISTINCT id FROM Nodes_tags \
WHERE value IS 'restaurant' OR value is 'fast_food' OR value is 'cafe';")
x = cur.fetchall()
for tup in x:
    food_ids.append(tup[0])

cur.execute("SELECT * FROM Nodes_tags WHERE key is 'name'")
y = cur.fetchall()
for tup in y:
    if tup[0] in food_ids:
        food_places[tup[2]] += 1

In [22]:
pprint.pprint(food_places)

defaultdict(<class 'int'>,
            {'10 Barrel Brewpub and Restaurant': 1,
             'Andrades': 1,
             'Applebees': 1,
             'Arbys': 1,
             'Artic Circle': 1,
             'Asian Wok Chinese': 1,
             'Bad Boy Burgers': 1,
             'Bardenay': 2,
             'Baskin Robbins Ice Cream': 1,
             'Baskin Robins': 1,
             'Ben n Jerrys Ice Cream': 1,
             'Big Bun': 1,
             'Big Juds': 1,
             'Blimpie': 4,
             'Boise Fry Company': 1,
             'Bonefish Grill': 1,
             'Brewforia Beer Market and Restaurant': 1,
             'Buffalo Wild Wings': 1,
             'Burger King': 2,
             'Cafe Zupas': 1,
             'Carinos Italian': 1,
             'Carls Jr': 3,
             'Carls Jr Green Burrito': 1,
             'Carls Junior': 1,
             'Carls Junior Green Burrito': 1,
             'Chapala Mexican Restaurant': 1,
             'Chapalas Mexican Grill': 1,
         

In [23]:
# These things seem to be in every Idaho town, no matter how scarcely populated.
food_places['Subway']

13

**Author's Note**

* Restaurant names definitely benefit from cleaning. There were six different instances of Carl's Jr that varied based on punctuation and the presence of a Green Burrito. Most restaurants had at least one mispelling or alias, not all of which was fixed by stripping out problem characters

* The data itself is severly lacking. Off the cuff, less than 50% of Southwest Idaho resturaunts are represented.

In [24]:
# Investigate street data
ways = []
ways_set = set()
cur.execute("SELECT value from Ways_Tags WHERE key IS 'street' UNION ALL \
SELECT value from Nodes_Tags WHERE key IS 'street'")
x = cur.fetchall()

# Could have run a second query with the DISTINCT keyword,
# but it seemed wasteful to put in a second for-loop
for tup in x:
    ways.append(tup[0])
    ways_set.add(tup[0])

print('Number of named streets in data: ' + str(len(ways)))
print('Number of unique street names in data: ' + str(len(ways_set)))
print('Ratio of unique street names to total street name count: ' + str(len(ways_set)/len(ways)))

Number of named streets in data: 3353
Number of unique street names in data: 357
Ratio of unique street names to total street name count: 0.10647181628392484


In [25]:
# Check that cleaning did take effect
g = audit_streets(ways_set)
pprint.pprint(g)

defaultdict(<class 'set'>,
            {'320': {'West Main Street STE 320'},
             'A': {'West Fairview Avenue A'},
             'Avenue': {'1021 Broadway Avenue',
                        'Arlington Avenue',
                        'Broadway Avenue',
                        'Creekside Avenue',
                        'East Boise Avenue',
                        'East Broadway Avenue',
                        'East Hawaii Avenue',
                        'East Maine Avenue',
                        'East Roosevelt Avenue',
                        'Fairview Avenue',
                        'Florida Avenue',
                        'Georgia Avenue',
                        'Hawkins Avenue',
                        'Iowa Avenue',
                        'Lake Lowell Avenue',
                        'Law Avenue',
                        'Lincoln Avenue',
                        'Manitou Avenue',
                        'McClure Avenue',
                        'Melrose Avenue',
     

                       'South Stanley Creek Place',
                       'West Escalade Place',
                       'West Shy Creek Place',
                       'West Swallowtail Place',
                       'Whisperwood Place'},
             'Road': {'110west Overland Road',
                      '12th Avenue Road',
                      'Cemetery Road',
                      'East Frontage Road',
                      'East Greenhurst Road',
                      'Emmett Road',
                      'Karcher Road',
                      'Lake Hazel Road',
                      'North Cole Road',
                      'North Curtis Road',
                      'North Eagle Road',
                      'North Kings Road',
                      'North Linder Road',
                      'North Locust Grove Road',
                      'North Robbins Road',
                      'North Star Road',
                      'Old Penitentiary Road',
                      'Overland Roa

**Author's Note**

For reasons unknown, there are 46 more street instances in the SQL table than were detected by the original audit_streets. I suspect this is due to the slight differences in how the information is collected. The change is minor enough that I'm unconcerned with its overall impact.

In [26]:
# Investigate postal data
post = defaultdict(int)
cur.execute("SELECT value from Ways_Tags WHERE key LIKE 'post%' UNION ALL \
SELECT value from Nodes_Tags WHERE key LIKE 'post%'")
x = cur.fetchall()

for tup in x:
    post[tup[0]] += 1
    
pprint.pprint(post)

defaultdict(<class 'int'>,
            {'184 83702': 1,
             '83605': 9,
             '83607': 2,
             '83616': 6,
             '83634': 6,
             '83642': 58,
             '83644': 3,
             '83646': 20,
             '83651': 600,
             '83669': 6,
             '83678': 1,
             '83686': 2237,
             '83687': 177,
             '83701': 1,
             '83702': 29,
             '83703': 7,
             '83704': 40,
             '83705': 31,
             '83706': 30,
             '83709': 19,
             '83712': 8,
             '83713': 5,
             '83714': 41,
             '83716': 4,
             '83735': 1})


In [27]:
# Investigate participation by year
cur.execute("SELECT timestamp FROM Nodes UNION ALL SELECT timestamp FROM Ways;")
x = cur.fetchall()

yearly_participation = defaultdict(int)
for item in x:
    dt = item[0].split('-')
    yearly_participation[dt[0]] += 1
    
pprint.pprint(yearly_participation)

defaultdict(<class 'int'>,
            {'2006': 235,
             '2007': 3753,
             '2008': 596,
             '2009': 129641,
             '2010': 10815,
             '2011': 2861,
             '2012': 7103,
             '2013': 41638,
             '2014': 19585,
             '2015': 41182,
             '2016': 27000,
             '2017': 30188})


In [28]:
# Total changes
t = 0
for val in yearly_participation.values():
    t += val

print(t)

314597


In [29]:
# Investigate participation by user, in 2009
cur.execute("SELECT user,timestamp FROM Nodes WHERE timestamp LIKE '2009%' \
UNION ALL SELECT user,timestamp FROM Ways WHERE timestamp LIKE '2009%'")
x = cur.fetchall()

# Aggregate users
user_participation = defaultdict(int)
for item in x:
    user_participation[item[0]] += 1

pprint.pprint(user_participation)

defaultdict(<class 'int'>,
            {'C Bentz': 34,
             'Chris Lawrence': 13357,
             'Minh Nguyen': 920,
             'Tony Speer': 5,
             'amillar': 17,
             'barkerid': 2349,
             'chokeyou': 3,
             'h4ck3rm1k3': 14,
             'haveaswiss': 14,
             'iandees': 70,
             'isotope': 14,
             'jrglasgow': 322,
             'linuxuser728': 2,
             'mll1013': 7,
             'nmixter': 290,
             'pdunn': 259,
             'woodpeck_fixbot': 111950,
             'xybot': 14})


**Author's Note**

Normal participation seems to fluctuate around the 10,000's, with a few years being exceptionally small or large. The year 2009 stands out as a significant outlier, with a bot called 'woodpeck_fixbot' making over 111,000 contributions.